In [1]:
import json

In [2]:
caption_eval_app_data_file = 'allresponses.json'
with open(caption_eval_app_data_file) as f:
    caption_eval_data = json.load(f)

In [3]:
total_rated = len(caption_eval_data)
print(caption_eval_data[1089])
print(caption_eval_data[4036])

{'Image': 'http://images.cocodataset.org/val2014/COCO_val2014_000000056344.jpg', 'Caption': 'a computer screen is shown with a tv and a computer .', 'CapID': 7773, 'Consensus': '3.0000', 'Z0': 3, 'Z1': 3}
{'Image': 'http://images.cocodataset.org/val2014/COCO_val2014_000000026323.jpg', 'Caption': 'A herd of giraffe walking across a grass covered field.', 'CapID': 27790, 'Consensus': '5.0000'}


In [14]:
cosine_sim_ratings = []
human_ratings = []

for obj in caption_eval_data:
    if 'Z0' in obj:
        cosine_rating = obj['Z0']
    elif 'Z1' in obj:
        cosine_rating = obj['Z1']
    else:
        cosine_rating = obj['Consensus']
    cosine_sim_ratings.append(cosine_rating)
    total_human_ratings = 0
    human_sum = 0
    z_val = 2
    while True:
        key = 'Z'+str(z_val)
        if key in obj:
            total_human_ratings += 1
            human_sum += int(obj[key])
            z_val += 1
        else:
            break
    
    if total_human_ratings > 0:
        human_average = human_sum / total_human_ratings
    else:
        human_average = -1
    
    human_ratings.append(human_average)

http://images.cocodataset.org/val2014/COCO_val2014_000000056344.jpga computer screen is shown with a tv and a computer .
http://images.cocodataset.org/val2014/COCO_val2014_000000081081.jpga bird is standing in the snow on a sunny day .
http://images.cocodataset.org/val2014/COCO_val2014_000000116951.jpga train is parked in front of a large building .
http://images.cocodataset.org/val2014/COCO_val2014_000000150576.jpga bird is standing in the grass with a large tree .
http://images.cocodataset.org/val2014/COCO_val2014_000000192894.jpga bird is standing in the grass with a large tree .
http://images.cocodataset.org/val2014/COCO_val2014_000000233005.jpga dog is sitting on a bed in a room .
http://images.cocodataset.org/val2014/COCO_val2014_000000252133.jpga sign that says `` [ male ] is on the side of a road .
http://images.cocodataset.org/val2014/COCO_val2014_000000359039.jpga group of people that are standing next to each other .
http://images.cocodataset.org/val2014/COCO_val2014_0000004

In [13]:
print(len(cosine_sim_ratings))
print(len(human_ratings))


12039
12039


In [6]:
count_of_no_cosine = 0
for idx, rat in enumerate(cosine_sim_ratings):
    if rat == -1:
        print(idx)
        count_of_no_cosine += 1
        
print('Number of samples without cosine similarity data = ', count_of_no_cosine)

count_of_no_human = 0
for rat in human_ratings:
    if rat == -1:
        count_of_no_human += 1
        
print('Number of samples without human data = ', count_of_no_human)

Number of samples without cosine similarity data =  0
Number of samples without human data =  1408


In [8]:
# Totals_by_rating means the total number of times
# Need confusion matrix? Can do if we assume human to be ground truth
# This is tricky because we did not use cosine similarity for known 1s (mismatched)
# Unclear which "totals" to take. The cosine similarity totals? or the human totals? 
totals_by_rating = [0, 0, 0, 0, 0, 0]
correct_by_rating = [0, 0, 0, 0, 0, 0]

total_with_cos_and_human = 0
total_correct = 0
for idx, (human, cos) in enumerate(zip(human_ratings, cosine_sim_ratings)):
    if human == -1 or cos == -1:
        continue
    if round(human) == int(cos):
        total_correct += 1
        correct_by_rating[int(cos)] += 1
    
    total_with_cos_and_human += 1
    #totals_by_rating[int(cos)] += 1
    totals_by_rating[round(human)] += 1
        
accuracy = total_correct / total_with_cos_and_human

print('Total with both cosine similarity and human ratings = ', total_with_cos_and_human)
print('Total exact match = ', total_correct)
print('Accuracy = ', accuracy)
print('---------------------------')
print('Totals by rating sum = ', sum(totals_by_rating))
print('Totals by rating = ', totals_by_rating)
print('Correct by rating = ', correct_by_rating)
print('Accuracy by rating 1 - 5')
for idx, (correct, total) in enumerate(zip(correct_by_rating, totals_by_rating)):
    if idx == 0:
        continue
    print('Rating = ', idx, '| Accuracy = ', correct/total)

Total with both cosine similarity and human ratings =  10631
Total exact match =  5271
Accuracy =  0.4958141284921456
---------------------------
Totals by rating sum =  10631
Totals by rating =  [0, 2316, 1895, 1335, 3656, 1429]
Correct by rating =  [0, 1761, 632, 482, 1721, 675]
Accuracy by rating 1 - 5
Rating =  1 | Accuracy =  0.7603626943005182
Rating =  2 | Accuracy =  0.33350923482849604
Rating =  3 | Accuracy =  0.3610486891385768
Rating =  4 | Accuracy =  0.47073304157549234
Rating =  5 | Accuracy =  0.4723582925122463


In [35]:
from sklearn.metrics import confusion_matrix